In [ ]:
### Data
import xarray as xr
import cfgrib
import numpy as np

### Plotting
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import matplotlib.ticker as mticker
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
import matplotlib.transforms as mtransforms

### Warnings
import warnings
warnings.filterwarnings('ignore')

def get_z(filepath, min_lat=-90, max_lat=90, min_lon=-180, max_lon=180):
    
    file = xr.open_dataset(filepath, engine='cfgrib',
                backend_kwargs={'filter_by_keys': {'typeOfLevel': 'isobaricInhPa','shortName': 'gh'}})
    var = file.get('gh')[12].to_dataframe()
    latitudes = var.index.get_level_values('latitude')
    longitudes = var.index.get_level_values('longitude')
    map_function = lambda lon: (lon - 360) if (lon > 180) else lon
    remapped_longitudes = longitudes.map(map_function)
    var['longitude'] = remapped_longitudes
    var['latitude'] = latitudes
    lat_filter = (var['latitude'] >= min_lat) & (var['latitude'] <= max_lat)
    lon_filter = (var['longitude'] >= min_lon) & (var['longitude'] <= max_lon)
    var = var.loc[lat_filter & lon_filter]
    var = var.set_index(['latitude', 'longitude']).to_xarray()
    
    return var

def get_slp(filepath, min_lat=-90, max_lat=90, min_lon=-180, max_lon=180):
    
    file = xr.open_dataset(filepath, engine='cfgrib',
                backend_kwargs={'filter_by_keys': {'typeOfLevel': 'meanSea','shortName': 'prmsl'}})
    var = file.get('prmsl').to_dataframe()
    latitudes = var.index.get_level_values('latitude')
    longitudes = var.index.get_level_values('longitude')
    map_function = lambda lon: (lon - 360) if (lon > 180) else lon
    remapped_longitudes = longitudes.map(map_function)
    var['longitude'] = remapped_longitudes
    var['latitude'] = latitudes
    lat_filter = (var['latitude'] >= min_lat) & (var['latitude'] <= max_lat)
    lon_filter = (var['longitude'] >= min_lon) & (var['longitude'] <= max_lon)
    var = var.loc[lat_filter & lon_filter]
    var = var.set_index(['latitude', 'longitude']).to_xarray()
    
    return var


In [ ]:
min_lat = 15
max_lat = 56
min_lon = -120
max_lon = -60

z1 = get_z('/p/work1/lloveras/nov2018/gfs_files/analysis/gfs.0p25.2018111312.f000.grib2',
                 min_lat, max_lat, min_lon, max_lon)
slp1 = get_slp('/p/work1/lloveras/nov2018/gfs_files/analysis/gfs.0p25.2018111312.f000.grib2',
                 min_lat, max_lat, min_lon, max_lon)

z2 = get_z('/p/work1/lloveras/nov2018/gfs_files/analysis/gfs.0p25.2018111412.f000.grib2',
                 min_lat, max_lat, min_lon, max_lon)
slp2 = get_slp('/p/work1/lloveras/nov2018/gfs_files/analysis/gfs.0p25.2018111412.f000.grib2',
                 min_lat, max_lat, min_lon, max_lon)

z3 = get_z('/p/work1/lloveras/nov2018/gfs_files/analysis/gfs.0p25.2018111512.f000.grib2',
                 min_lat, max_lat, min_lon, max_lon)
slp3 = get_slp('/p/work1/lloveras/nov2018/gfs_files/analysis/gfs.0p25.2018111512.f000.grib2',
                 min_lat, max_lat, min_lon, max_lon)

z4 = get_z('/p/work1/lloveras/nov2018/gfs_files/analysis/gfs.0p25.2018111612.f000.grib2',
                 min_lat, max_lat, min_lon, max_lon)
slp4 = get_slp('/p/work1/lloveras/nov2018/gfs_files/analysis/gfs.0p25.2018111612.f000.grib2',
                 min_lat, max_lat, min_lon, max_lon)


In [ ]:
fig, axd = plt.subplot_mosaic([['topleft','topright','cbar'],['bottomleft','bottomright','cbar']],
                              constrained_layout=True, figsize=(8.2,5.4), dpi=200, 
                              gridspec_kw={'width_ratios':[1,1,0.05],'height_ratios':[1,1]},
                              per_subplot_kw={'topleft':{'projection':crs.PlateCarree()},
                                              'topright':{'projection':crs.PlateCarree()},
                                              'bottomleft':{'projection':crs.PlateCarree()},
                                              'bottomright':{'projection':crs.PlateCarree()}})

trans = mtransforms.ScaledTranslation(1/30, -1/30, fig.dpi_scale_trans)

##############
### TOP LEFT
#############

axd['topleft'].set_extent((min_lon,max_lon,min_lat,max_lat-4))

axd['topleft'].contour(z1['longitude'], z1['latitude'], z1['gh']/10.,
                  levels=np.arange(500,610,10),extend='both',alpha=0.5,
                  colors='white', transform=crs.PlateCarree())
im1 = axd['topleft'].contourf(z1['longitude'], z1['latitude'], z1['gh']/10.,
                  levels=np.arange(500,610,10),extend='both',
                  cmap=get_cmap('turbo'), transform=crs.PlateCarree())

cs1 = axd['topleft'].contour(slp1['longitude'], slp1['latitude'], slp1['prmsl']/100.,
                  levels=np.arange(900,1060,4),
                  colors='k', transform=crs.PlateCarree())
axd['topleft'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(900,1060,8),fontsize=6,colors='k')

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['topleft'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['topleft'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Add the gridlines
gls = axd['topleft'].gridlines(draw_labels=True, dms=True,
                   x_inline=False, y_inline=False,linestyle='dashed')
gls.top_labels=False
gls.bottom_labels=False
gls.right_labels=False
gls.left_labels=True
gls.xlocator = mticker.FixedLocator([-110, -90, -70])
gls.ylocator = mticker.FixedLocator([20, 30, 40, 50])

axd['topleft'].set_title('1200 UTC 13 Nov 2018')
axd['topleft'].text(0.0, 1.0, '(a)',transform=axd['topleft'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### TOP RIGHT
#############

axd['topright'].set_extent((min_lon,max_lon,min_lat,max_lat-4))

axd['topright'].contour(z2['longitude'], z2['latitude'], z2['gh']/10.,
                  levels=np.arange(500,610,10),extend='both',alpha=0.5,
                  colors='white', transform=crs.PlateCarree())
im1 = axd['topright'].contourf(z2['longitude'], z2['latitude'], z2['gh']/10.,
                  levels=np.arange(500,610,10),extend='both',
                  cmap=get_cmap('turbo'), transform=crs.PlateCarree())


cs1 = axd['topright'].contour(slp2['longitude'], slp2['latitude'], slp2['prmsl']/100.,
                  levels=np.arange(900,1060,4),
                  colors='k', transform=crs.PlateCarree())
axd['topright'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(900,1060,8),fontsize=6,colors='k')

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['topright'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['topright'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Add the gridlines
gls = axd['topright'].gridlines(draw_labels=True, dms=True,
                   x_inline=False, y_inline=False,linestyle='dashed')
gls.top_labels=False
gls.bottom_labels=False
gls.right_labels=False
gls.left_labels=False
gls.xlocator = mticker.FixedLocator([-110, -90, -70])
gls.ylocator = mticker.FixedLocator([20, 30, 40, 50])

axd['topright'].set_title('1200 UTC 14 Nov 2018')
axd['topright'].text(0.0, 1.0, '(b)',transform=axd['topright'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### BOTTOM LEFT
#############

axd['bottomleft'].set_extent((min_lon,max_lon,min_lat,max_lat-4))

axd['bottomleft'].contour(z3['longitude'], z3['latitude'], z3['gh']/10.,
                  levels=np.arange(500,610,10),extend='both',alpha=0.5,
                  colors='white', transform=crs.PlateCarree())
im1 = axd['bottomleft'].contourf(z3['longitude'], z3['latitude'], z3['gh']/10.,
                  levels=np.arange(500,610,10),extend='both',
                  cmap=get_cmap('turbo'), transform=crs.PlateCarree())

cs1 = axd['bottomleft'].contour(slp3['longitude'], slp3['latitude'], slp3['prmsl']/100.,
                  levels=np.arange(900,1060,4),
                  colors='k', transform=crs.PlateCarree())
axd['bottomleft'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(900,1060,8),fontsize=6,colors='k')

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['bottomleft'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['bottomleft'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Add the gridlines
gls = axd['bottomleft'].gridlines(draw_labels=True, dms=True,
                   x_inline=False, y_inline=False,linestyle='dashed')
gls.top_labels=False
gls.bottom_labels=True
gls.right_labels=False
gls.left_labels=True
gls.xlocator = mticker.FixedLocator([-110, -90, -70])
gls.ylocator = mticker.FixedLocator([20, 30, 40, 50])

axd['bottomleft'].set_title('1200 UTC 15 Nov 2018')
axd['bottomleft'].text(0.0, 1.0, '(c)',transform=axd['bottomleft'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)


##############
### BOTTOM RIGHT
#############

axd['bottomright'].set_extent((min_lon,max_lon,min_lat,max_lat-4))

axd['bottomright'].contour(z4['longitude'], z4['latitude'], z4['gh']/10.,
                  levels=np.arange(500,610,10),extend='both',alpha=0.5,
                  colors='white', transform=crs.PlateCarree())
im1 = axd['bottomright'].contourf(z4['longitude'], z4['latitude'], z4['gh']/10.,
                  levels=np.arange(500,610,10),extend='both',
                  cmap=get_cmap('turbo'), transform=crs.PlateCarree())

cs1 = axd['bottomright'].contour(slp4['longitude'], slp4['latitude'], slp4['prmsl']/100.,
                  levels=np.arange(900,1060,4),
                  colors='k', transform=crs.PlateCarree())
axd['bottomright'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(900,1060,8),fontsize=6,colors='k')

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['bottomright'].add_feature(states, linewidth=0.6, edgecolor='k',alpha=0.5)
axd['bottomright'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Add the gridlines
gls = axd['bottomright'].gridlines(draw_labels=True, dms=True,
                   x_inline=False, y_inline=False,linestyle='dashed')
gls.top_labels=False
gls.bottom_labels=True
gls.right_labels=False
gls.left_labels=False
gls.xlocator = mticker.FixedLocator([-110, -90, -70])
gls.ylocator = mticker.FixedLocator([20, 30, 40, 50])

axd['bottomright'].set_title('1200 UTC 16 Nov 2018')
axd['bottomright'].text(0.0, 1.0, '(d)',transform=axd['bottomright'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

### SET THE COLORBAR AND SHOW
cbar = fig.colorbar(im1, orientation='vertical', cax=axd['cbar'])

plt.savefig('/p/work1/lloveras/nov2018/figs_pdf/fig01_gfs_overview.pdf',bbox_inches='tight')
plt.show()
